In [5]:
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 
import re
import tweepy
import mysql.connector #To connect to mysql
import pandas as pd
from textblob import TextBlob

In [6]:
class MyStreamListener(tweepy.StreamListener): #Class for streaming
    
    def on_status(self, status): #Extract tweets text
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        #Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        
        if status_code == 420:
            # return False to disconnect the stream
            return False

In [7]:
def clean_tweet(self, tweet): 
    #Use sumple regex statemnents to clean tweet text by removing links and special characters

    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    #Strip all non-ASCII characters to remove emoji characters

    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [ ]:
mydb = mysql.connector.connect(
    host="localhost", #127.0.0.1
    user="user",
    passwd="password",
    database="database",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

In [9]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOEKN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)

mydb.close()

In [ ]:
mydb.close()

In [15]:
mydb

In [16]:
db_connection = mysql.connector.connect(
    host="localhost", #127.0.0.1
    user="user",
    passwd="password",
    database="database",
    charset = 'utf8'
)

In [17]:
db_connection

In [18]:
df = pd.read_sql('SELECT id_str, text, created_at, polarity, user_location FROM {}'.format(settings.TABLE_NAME), con=db_connection)

In [19]:
df #Tweets collected

,id_str,text,created_at,polarity,user_location
0,1333670422106304512,RT @bioethicsdotcom: Absolutely Remarkable: No...,2020-12-01 07:13:01,0,"Madrid, Spain"
1,1333670422135644162,RT @BTSFanQuiz: The fifth track of the BE albu...,2020-12-01 07:13:01,0,Albania
2,1333670422299152387,"RT @KailashChandOBE: Dear @RishiSunak ,\nCorre...",2020-12-01 07:13:01,0,"Ross-on-Wye, England"
3,1333670422508924928,RT @EmilyThornberry: Deeply moving report on P...,2020-12-01 07:13:01,0,None
4,1333670422529912833,RT @thebradfordfile: Even CNN is now admitting...,2020-12-01 07:13:01,0,None
...,...,...,...,...,...
19358,1333906648411344897,RT @kheti_thegreat: Covid has taught me three ...,2020-12-01 22:51:42,-1,On My Way...
19359,1333906648541274113,RT @Milhouse_Van_Ho: Canada - The average age ...,2020-12-01 22:51:42,0,"Langley, BC, Canada"
19360,1333906648700620800,RT @AlexBerenson: How COVID ruined my life - f...,2020-12-01 22:51:42,0,None
19361,1333906648801435651,RT @campbellclaret: A reminder that the people...,2020-12-01 22:51:42,0,Cheshire.UK


In [24]:
df.to_csv('tweets2020.csv') #export